In [1]:
#Import various useful libraries
import pandas as pd
import numpy as np

import datetime as dt
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
from sqlalchemy import create_engine

In [2]:
sql_con = "sqlite:///../database/ufc_data.db"

In [3]:
con = create_engine(sql_con)
fighter_df = pd.read_sql("SELECT * FROM fighter_data", con)

In [6]:
def clean_fighter_data(sql_con,fighter_df):
    fighter_df = fighter_df.astype({'stance':'category'})
    fighter_df['record'] = fighter_df.record.str.replace('Record:',"")
    fighter_df['record'] = fighter_df.record.str.replace("\(.*\)","",regex=True)
    fighter_df['record'] = fighter_df.record.str.strip()
    
    fighter_df = pd.concat([fighter_df,fighter_df.record.str.split("-",n=2,expand=True).rename(columns={0:'wins',1:'losses',2:'draws'})],axis='columns').drop(columns='record')
    fighter_df['height'] = fighter_df.height.str.replace('"',"")
    heights = fighter_df.height.str.split("'",n=1,expand=True).rename(columns={0:'feet',1:'inches'})
    
    heights['feet'] = heights.feet.replace('--',np.nan)
    heights = heights.astype({'feet':'float32','inches':'float32'})
    
    fighter_df['height'] = ((heights.feet * 12) + heights.inches)*2.54
    fighter_df['weight']  = fighter_df.weight.str.replace("lbs.","")
    fighter_df['weight'] = fighter_df['weight'].replace('--',np.nan)
    fighter_df['weight'] = fighter_df['weight'].astype('float32') / 2.20462
    
    fighter_df['reach'] = fighter_df['reach'].replace('--',np.nan)
    fighter_df['reach'] = fighter_df['reach'].replace('"',"",regex=True)
    fighter_df['reach'] = fighter_df['reach'].astype('float32') * 2.54
    
    fighter_df['stance'] = fighter_df.stance.replace({'Orthodox':1,'Southpaw':10,'Switch':100,'Open Stance':1000,'Sideways':10000,"":1}) #Note - makes unknowns into Orthodox
    
    year = fighter_df.dob.str.split(" ",n=1,expand=True)[1].str.split(",",expand=True)[1]
    
    fighter_df['dob'] = fighter_df.dob.str.split(" ",n=1,expand=True)[1].str.split(",",expand=True)[1]
    
    fighter_df['strike_accuracy'] = fighter_df['strike_accuracy'].replace("%","",regex=True).astype('float32')
    fighter_df['strike_defense'] = fighter_df['strike_defense'].replace("%","",regex=True).astype('float32')
    fighter_df['takedown_accuracy'] = fighter_df['takedown_accuracy'].replace("%","",regex=True).astype('float32')
    fighter_df['takedown_defense'] = fighter_df['takedown_defense'].replace("%","",regex=True).astype('float32')
    
    fighter_df = fighter_df.astype({'dob':'float32','wins':'int32','losses':'int32','draws':'int32','strikes_landed_per_min':'float','stance':'int','strikes_absorbed_per_min':'float',
                                   'takedowns_per_15_min':'float','submission_attempts_per_15_min':'float'})
    
    fighter_df['total_fights'] = fighter_df.wins + fighter_df.losses + fighter_df.draws
    
    fighter_df['win_pct'] = fighter_df.wins / fighter_df.total_fights
    fighter_df['loss_pct'] = fighter_df.losses / fighter_df.total_fights
    fighter_df['draw_pct'] = fighter_df.draws / fighter_df.total_fights
    
    fighter_df.to_sql('clean_fighter_data',con,if_exists='replace',index=False)

In [7]:
clean_fighter_data(sql_con,fighter_df)